In [12]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time
from collections import OrderedDict
# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


==> Preparing data..


In [13]:
transform_test

Compose(
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
)

In [34]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        
        self.convnet = nn.Sequential(OrderedDict([
            ('c1', nn.Conv2d(3, 3, kernel_size=(5, 5))),
            ('relu1', nn.ReLU()),
            ('s2', nn.MaxPool2d(kernel_size=(2, 2), stride=2)),
            ('c3', nn.Conv2d(3, 16, kernel_size=(5, 5))),
            ('relu3', nn.ReLU()),
            ('s4', nn.MaxPool2d(kernel_size=(2, 2), stride=2)),
            ('c5', nn.Conv2d(16, 120, kernel_size=(5, 5))),
            ('relu5', nn.ReLU())
        ]))

        self.fc = nn.Sequential(OrderedDict([
            ('f6', nn.Linear(120, 84)),
            ('relu6', nn.ReLU()),
            ('f7', nn.Linear(84, 10)),
            ('sig7', nn.LogSoftmax(dim=-1))
        ]))
        
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        
        output = self.convnet(x)
        output = output.view(x.size(0), -1)
        output = self.fc(output)
        
        ###########################
        #### End of your codes ####
        ###########################
    

        return output



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [35]:

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [39]:

def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 50
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()


Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.311004
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.303591
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.288186
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.217399
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.166404
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.159452
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.991313
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.148057
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.897762
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.004825
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.009946
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.857703
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.918488
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.898615
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.850434
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.904829
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.857255
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.883547
Train Epoch: 1 [23040/50000 (46%)]	Loss


Test set: Average loss: 1.4205, Accuracy: 4970/10000 (50%)

Train Epoch: 5 [0/50000 (0%)]	Loss: 1.459113
Train Epoch: 5 [1280/50000 (3%)]	Loss: 1.465025
Train Epoch: 5 [2560/50000 (5%)]	Loss: 1.458680
Train Epoch: 5 [3840/50000 (8%)]	Loss: 1.516639
Train Epoch: 5 [5120/50000 (10%)]	Loss: 1.630658
Train Epoch: 5 [6400/50000 (13%)]	Loss: 1.473494
Train Epoch: 5 [7680/50000 (15%)]	Loss: 1.677041
Train Epoch: 5 [8960/50000 (18%)]	Loss: 1.435045
Train Epoch: 5 [10240/50000 (20%)]	Loss: 1.460568
Train Epoch: 5 [11520/50000 (23%)]	Loss: 1.298710
Train Epoch: 5 [12800/50000 (26%)]	Loss: 1.591605
Train Epoch: 5 [14080/50000 (28%)]	Loss: 1.513321
Train Epoch: 5 [15360/50000 (31%)]	Loss: 1.481442
Train Epoch: 5 [16640/50000 (33%)]	Loss: 1.516635
Train Epoch: 5 [17920/50000 (36%)]	Loss: 1.760884
Train Epoch: 5 [19200/50000 (38%)]	Loss: 1.585333
Train Epoch: 5 [20480/50000 (41%)]	Loss: 1.398991
Train Epoch: 5 [21760/50000 (43%)]	Loss: 1.553396
Train Epoch: 5 [23040/50000 (46%)]	Loss: 1.612370
Trai

Train Epoch: 9 [1280/50000 (3%)]	Loss: 1.420969
Train Epoch: 9 [2560/50000 (5%)]	Loss: 1.521347
Train Epoch: 9 [3840/50000 (8%)]	Loss: 1.449654
Train Epoch: 9 [5120/50000 (10%)]	Loss: 1.359341
Train Epoch: 9 [6400/50000 (13%)]	Loss: 1.448230
Train Epoch: 9 [7680/50000 (15%)]	Loss: 1.407997
Train Epoch: 9 [8960/50000 (18%)]	Loss: 1.411573
Train Epoch: 9 [10240/50000 (20%)]	Loss: 1.450413
Train Epoch: 9 [11520/50000 (23%)]	Loss: 1.372007
Train Epoch: 9 [12800/50000 (26%)]	Loss: 1.399669
Train Epoch: 9 [14080/50000 (28%)]	Loss: 1.217958
Train Epoch: 9 [15360/50000 (31%)]	Loss: 1.436423
Train Epoch: 9 [16640/50000 (33%)]	Loss: 1.462525
Train Epoch: 9 [17920/50000 (36%)]	Loss: 1.270680
Train Epoch: 9 [19200/50000 (38%)]	Loss: 1.365312
Train Epoch: 9 [20480/50000 (41%)]	Loss: 1.518053
Train Epoch: 9 [21760/50000 (43%)]	Loss: 1.338441
Train Epoch: 9 [23040/50000 (46%)]	Loss: 1.384538
Train Epoch: 9 [24320/50000 (49%)]	Loss: 1.299989
Train Epoch: 9 [25600/50000 (51%)]	Loss: 1.638518
Train Epoc

KeyboardInterrupt: 